In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

npo_jobs = {}
job_no = 0

while True:
    url = "https://boston.craigslist.org/search/npo"
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    jobs = soup.find_all('div', {'class':'result-info'})
    for job in jobs:
        title = job.find('a', {'class':'result-title'}).text
        location_tag = job.find('span', {'class':'result-hood'})
        location = location_tag.text[2:-1] if location_tag else 'N/A'
        date = job.find('time', {'class':'result-date'}).text
        link = job.find('a', {'class':'result-title'}).get('href')
        job_response = requests.get(link)
        job_data = job_response.text
        job_soup = BeautifulSoup(job_data, 'html.parser')
        job_decription = job_soup.find('section', {'id':'postingbody'}).text
        job_attributes_tag = job_soup.find('p', {'class', 'attrgroup'})
        job_attributes = job_attributes_tag.text if job_attributes_tag else 'N/A'
        job_no+=1
        npo_jobs[job_no] = [title, location, date, link, job_decription, job_attributes]
        #print('Job Title:', title, '\nLocation:', location, '\nDate:', date, '\nLink:', link, '\nJob Description:', job_decription, '\nJob Attributes', job_attributes,  '\n-----')
    nextpageurl_tag = soup.find('a', {'title':'next page'})
    if nextpageurl_tag.get('href'):
        url = 'https://boston.craigslist.org' + nextpageurl_tag.get('href')
        print(url)
    else:
        break
        
print('Total Jobs:',job_no)
npo_jobs_df = pd.DataFrame.from_dict(npo_jobs, orient = 'index', columns = ['Job Title', 'Location', 'Date', 'Link', 'Job Decription', 'Job Attributes'])
npo_jobs_df.head(10)
npo_jobs_df.to_csv('npo_jobs.csv')

<a class="button next" href="" title="next page">next &gt; </a>
Total Jobs: 120
